# **Introducción al análisis de datos en Python** 
#### Profesor: Juan Pablo Salas
## Clase 6. Visualizaciones y repaso de Pandas

### Introducción a la libreria seaborn

In [167]:
import pandas as pd
pd.set_option('future.no_silent_downcasting', True)
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="darkgrid")

A continuación trabajaremos con algunos datos de propiedades de finca raíz en Colombia. Estos fueron extraidos de una página de clasificados donde cada usuario podía poner anuncios de venta o arriendo de distintos tipos de propiedades. Las columnas relevantes con las que se cuenta en esta base son:

- `start_date` y `end_date`: La fecha de inicio y de fin de la publicación
- `created_on`: La fecha en la que se realizó la publicación
- `lat` y `lon`: Las coordenadas geográficas de la propiedad ofertada
- `l1`: EL país de la propiedad ofertada
- `l2`: EL departamento/estado de la propiedad ofertada
- `l3`: La ciudad/el municipio de la propiedad ofertada
- `l4`: La zona de la propiedad ofertada
- `l5`: La localidad de la propiedad ofertada
- `l6`: El barrio de la propiedad ofertada
- `rooms`: El número de cuartos de la propiedad
- `bedrooms`: El número de habitaciones de la propiedad
- `bathrooms`: El número de baños de la propiedad
- `surface_total`: El área total construida en $m^2$ del a propiedad
- `surface_covered`: El área total del predio en $m^2$ del a propiedad
- `price`: El precio de la propiedad (recordar: mil millones son $10^9$)
- `currency`: La moneda en la que está demarcado el precio
- `property_type`: El tipo de propiedad (Apartamento, casa, lote, etc.)
- `operation_type`: El tipo de operación (venta, arriendo, etc.)

In [168]:
propiedades = pd.read_csv('./co_properties.csv')

In [ ]:
propiedades.head()

In [170]:
pd.set_option('display.max_columns', 26)

In [171]:
propiedades_venta_cali = propiedades[(propiedades.l3=='Cali') &(propiedades.operation_type=='Venta')&(propiedades.currency=='COP')]

#### Diagrama de dispersión y distribución (jointplot) de precio y superficie total

In [ ]:
sns.jointplot(data=propiedades_venta_cali,x='surface_total',y='price',xlim=(0,400),ylim=(0,1e9),height=10,hue='property_type')

#### Mapa de calor de número de baños por sector (heatmap)

Quisieramos entender cómo fluctúa el precio en función de dos variables: el sector (`l4`) y el número de baños (`bathrooms`) de propiedades de vivienda.

In [ ]:
# Filtramos por Casa, Apartamento y nos aseguramos de eliminar los valores nulos
propiedades_venta_cali.loc[propiedades_venta_cali.property_type.isin(['Casa','Apartamento']),['bathrooms','l4','price']].dropna()

In [ ]:
# Calculamos el precio por medio por barrio y número de baños
propiedades_venta_cali.loc[propiedades_venta_cali.property_type.isin(['Casa','Apartamento']),['bathrooms','l4','price']] \
    .dropna() \
        .groupby(['l4','bathrooms']).mean()

In [ ]:
propiedades_venta_cali.loc[propiedades_venta_cali.property_type.isin(['Casa','Apartamento']),['bathrooms','l4','price']] \
    .dropna() \
        .groupby(['l4','bathrooms']).mean().reset_index()

In [ ]:
# Construimos la tabla pivote
bathrooms_sector = propiedades_venta_cali.loc[propiedades_venta_cali.property_type.isin(['Casa','Apartamento']),['bathrooms','l4','price']] \
    .dropna() \
        .groupby(['l4','bathrooms']).mean().reset_index() \
        .pivot(index='l4',columns='bathrooms',values='price')
bathrooms_sector

In [ ]:
sns.heatmap(bathrooms_sector,cmap=sns.color_palette('berlin'),annot=True)

In [ ]:
plt.figure(figsize = (20,5))
sns.heatmap(bathrooms_sector,cmap=sns.color_palette('berlin'),annot=True,fmt='.1e')
plt.show ()

In [ ]:
plt.figure(figsize = (15,5))
sns.heatmap(bathrooms_sector[bathrooms_sector.columns[bathrooms_sector.columns<8]]/1000000,cmap=sns.color_palette('managua'),annot=True,fmt='.1f')
plt.title('Precio (en millones de pesos) de propiedades residenciales en Cali por barrio y número de baños')
plt.xlabel('Baños')
plt.ylabel('Barrios')
plt.show ()

### Introducción a la librería `plotly`

In [180]:
#!pip install plotly

In [181]:
import plotly.express as px

#### Gráficos de barra

Ahora queremos entender como cambia el precio promedio de los arriendos de algún tipo de propiedad con el tiempo. Para eso, usaremos las columnas `created_on`, `l3`,`surface_total` y `property_type` de nuestro DataFrame.

In [ ]:
propiedades.head()

In [183]:
propiedades['created_on'] = pd.to_datetime(propiedades['created_on'])

In [ ]:
arriendos_med = propiedades.loc[(propiedades.l3=='Medellín')&\
                (propiedades.property_type=='Casa')&\
                (propiedades.currency=='COP')&\
                    (propiedades.operation_type=='Arriendo'),['created_on','surface_covered','price']].dropna(subset='surface_covered')
arriendos_med

In [185]:
arriendos_med['mes'] = arriendos_med['created_on'].apply(lambda x: f'{x.year}-{x.month:02}')
arriendos_med = arriendos_med.groupby('mes').mean(numeric_only=True).reset_index()

In [ ]:
fig = px.bar(arriendos_med,x='mes',y='price',hover_data=['surface_covered'],color='surface_covered'\
    ,title='Precios promedio de arriendos en casas en Medellín',labels={'price':'Precio mensual'})
fig.show()


#### Gráficos de burbuja

In [ ]:
datos_burbuja = propiedades[(propiedades.l3=='Bogotá D.C')&(propiedades.price<1e10)&(propiedades.currency=='COP')].dropna(subset='bedrooms')
datos_burbuja.head()

In [ ]:
fig = px.scatter(datos_burbuja, x="surface_total", y="price",
	         size="bedrooms", color="property_type",hover_name='title',hover_data=['bedrooms','property_type'],
             log_y=True,log_x=True, size_max=60,
             title='Precio de propiedades en Bogotá en función del área superficial y el número de habitaciones.',
             labels={'price':'Precio (millones de pesos)','surface_total':'Área superficial (metros cuadrados)'})
fig.show()

### Introducción a la librería `geopandas`

In [189]:
#!pip install geopandas

In [190]:
import geopandas as gpd

#### Mapas de producción de café en Colombia**

In [191]:
area_cafe = pd.read_excel('./area_cult_tecnificacion.xlsx')

In [ ]:
area_cafe.head()

In [193]:
area_cafe = pd.read_excel('./area_cult_tecnificacion.xlsx',skiprows=5,header=[0,1])

In [ ]:
area_cafe.head()

In [ ]:
area_cafe = area_cafe.drop(columns='Unnamed: 0_level_0')

In [196]:
area_cafe = area_cafe.iloc[:23]

In [ ]:
area_cafe = area_cafe.set_index('Departamento')
area_cafe.head()

In [ ]:
area_cafe.unstack().reset_index()

In [199]:
area_cafe = area_cafe.unstack().reset_index().rename(columns={'level_0':'Tecnificación','level_1':'Año',0:'Área (kHa)'})

In [ ]:
area_cafe

In [201]:
area_cafe = area_cafe[area_cafe.Tecnificación!='Total']
area_cafe.loc[:,'Tecnificación'] = area_cafe.Tecnificación.apply(lambda s: s.split(' /')[0])
area_cafe.loc[:,'Departamento'] = area_cafe.Departamento.apply(lambda tupla: tupla[0].upper())
area_cafe.loc[:,'Año'] = area_cafe.Año.apply(lambda a: int(str(a).replace('*','')))
area_cafe.loc[:,'Área (kHa)'] = pd.to_numeric(area_cafe['Área (kHa)'])

In [ ]:
area_cafe = area_cafe[(area_cafe.Año==2007) | (area_cafe.Año==2022)]
area_cafe

In [203]:
info_geografica = gpd.read_file('./colombia_deptos.json')

In [ ]:
info_geografica.head()

In [ ]:
info_geografica.plot()

In [ ]:
area_cafe_2007_trad = info_geografica[['NOMBRE_DPT','geometry']].merge(area_cafe[(area_cafe.Año==2007) & (area_cafe.Tecnificación=='Tradicional')],left_on='NOMBRE_DPT',right_on='Departamento',how='left')
print(area_cafe_2007_trad.shape)
area_cafe_2007_trad.head()

In [ ]:
area_cafe_2007_trad.loc[:,'Tecnificación'] = area_cafe_2007_trad['Tecnificación'].fillna('Tradicional')
area_cafe_2007_trad.loc[:,'Año'] = area_cafe_2007_trad['Año'].fillna(2007)
area_cafe_2007_trad.loc[:,'Departamento'] = area_cafe_2007_trad['Departamento'].fillna(area_cafe_2007_trad.loc[:,'NOMBRE_DPT'])
area_cafe_2007_trad.loc[:,'Área (kHa)'] = area_cafe_2007_trad['Área (kHa)'].fillna(0)

In [ ]:
fig = plt.figure()
area_cafe_2007_trad.plot(column='Área (kHa)',cmap='Oranges')
plt.title('Área (kHa) de cultivos de café de \n tecnificado Tradicional en al año 2007')

In [ ]:
fig, axs = plt.subplots(2,3,figsize=(15,10),sharex=True,sharey=True,subplot_kw={'aspect':'equal'})

for año in area_cafe.Año.unique():
    for tec in area_cafe.Tecnificación.unique():
        ax = axs[list(area_cafe.Año.unique()).index(año),list(area_cafe.Tecnificación.unique()).index(tec)]
        area_cafe_subset = info_geografica[['NOMBRE_DPT','geometry']]\
            .merge(area_cafe[(area_cafe.Año==año) & (area_cafe.Tecnificación==tec)],\
                left_on='NOMBRE_DPT',right_on='Departamento',how='left')
        

        area_cafe_subset.loc[:,'Tecnificación'] = area_cafe_subset['Tecnificación'].fillna(tec)
        area_cafe_subset.loc[:,'Año'] = area_cafe_subset['Año'].fillna(año)
        area_cafe_subset.loc[:,'Departamento'] = area_cafe_subset['Departamento'].fillna(area_cafe_subset.loc[:,'NOMBRE_DPT'])
        area_cafe_subset.loc[:,'Área (kHa)'] = area_cafe_subset['Área (kHa)'].fillna(0)
        area_cafe_subset = area_cafe_subset.infer_objects()

        area_cafe_subset.plot(column='Área (kHa)',cmap='Oranges', ax=ax,edgecolor='#542201',linewidth=0.35)
        ax.grid(False)
        ax.set_facecolor('white')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
        ax.set_title(f'{tec}, {año}.')                

patch_col = axs[0][0].collections[0]
cb = fig.colorbar(patch_col, ax=axs, shrink=0.5)

## Taller

In [210]:
import pandas as pd
import numpy as np
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

El Departamento de Transporte de Estados Unidos rastrea el rendimiento de los vuelos domésticos operados por las principales transportadoras aéreas del país. A continuación, se presentan los datos de los vuelos a tiempo, retrasados, cancelados y desvíados para el año 2015.

Primero, importemos los dataframes y hagamos una exploración.

In [211]:
aerolineas = pd.read_csv('./airlines.csv')

In [ ]:
aerolineas.head()

Este parece ser un diccionario de las aerolíneas y su código único IATA (Asociación Internacional de Transporte Aéreo).

In [ ]:
aeropuertos = pd.read_csv('./airports.csv')
print(aeropuertos.shape)
aeropuertos.head()

En este caso tenemos la información de 322 aeropuertos como su código IATA, su nombre completo, la ciudad y el estado al que pertenecen así como sus coordenadas geográficas.

In [ ]:
vuelos = pd.read_csv('./flights.csv')
print(vuelos.shape)
vuelos.head()

Observe que tenemos 5,819,079 registros y aún así el tiempo de lectura fue corto. No obstante, como tenemos 31 columnas no es tan fácil visualizar toda la tabla. Para esto podemos cambiar el máximo número de columnas visualizadas utilizando:

In [215]:
pd.set_option('display.max_columns', 32)

In [ ]:
vuelos.head()

Veamos algunas descripciones de las columnas de los datos:

- **YEAR, MONTH, DAY**: Información sobre la fecha del vuelo.
- **AIRLINE**: Código IATA de la aerolínea.
- **FLIGHT_NUMBER**: Número del vuelo
- **TAIL_NUMBER**: Modelo del avión
- **ORIGIN_AIRPORT, DESTINATION_AIRPORT**: Código IATA del aeropuerto de origen y de destino respectivamente
- **SCHEDULED_DEPARTURE**: Hora del vuelo programada para la salida en formato 24H, es decir HHMM.
- **DEPARTURE_TIME**: Hora del vuelo real de la salida en formato 24H, es decir HHMM.
- **WHEELS OFF**: La hora a la que las llantas despegan. En format 24H (HHMM).
- **SCHEDULED_TIME**: El tiempo programado para la duración del vuelo.
- **AIR_TIME**: El tiempo que la aeronave estuvo en el aire en minutos.
- **DISTANCE**: La distancia recorrida en millas.
- **WHEELS_ON**: La hora a la que las llantas aterrizan. En format 24H (HHMM).
- **SCHEDULED_ARRIVAL**: Hora del vuelo programada para la llegada en formato 24H, es decir HHMM.
- **ARRIVAL_TIME**: Hora del vuelo real de la llegada en formato 24H, es decir HHMM.
- **DIVERTED, CANCELLED**: Si el vuelo fue desvíado o cancelado.
- **CANCELLATION_REASON**: Si el vuelo fue cancelado, indica la razón de su cancelación. Puede ser
    - _A_: Debido a problemas con la aerolínea.
    - _B_: Debido al clima.
    - _C_: Debido a restricciones del espacio aéreo (NAS)
    - _D_: Por razones de seguridad.
- **AIR_SYSTEM_DELAY, SECURITY_DELAY, AIRLINE_DELAY, LATE_AIRCRAFT_DELAY, WEATHER_DELAY**: Demoras en minutos debido al espacio aéreo, a la seguridad del viaje, a la aerolínea, a una aeronave tardía, al clima respectivamente.


1. ¿Cuál es el día de la semana dónde hay más vuelos retrasados? ¿Y más vuelos cancelados?

In [218]:
dict_day = {0: 'Lunes', 1:'Martes', 2:'Miércoles',3:'Jueves',4:'Viernes',5:'Sábado',6:'Domingo'}

In [219]:
vuelos['DIA_DE_SEMANA'] = pd.to_datetime(vuelos[['YEAR','MONTH','DAY']]).apply(lambda d: dict_day[d.day_of_week])

2. ¿Cuál es la aerolínea con más retrasos superiores a 45 min? ¿Y cancelaciones? Construya un gráfico de barras con el total de retrasos superiores a 45 min y cancelaciones por aerolínea?

3. Construya las distribuciones de la velocidad promedio (km/h) y la distancia (km) recorrida en los vuelos? ¿Cuál es la mediana de la velocidad promedio?

In [220]:
vuelos['DISTANCE_KM'] = vuelos['DISTANCE']*1.60934

In [221]:
vuelos['VELOCIDAD_PROMEDIO'] = vuelos['DISTANCE_KM']/(vuelos['AIR_TIME']/60)

4. Construya el mapa de calor de correlaciones para las variables de *VELOCIDAD_PROMEDIO, DISTANCE, AIR_TIME, TAXI_OFF, DEPARTURE_DELAY, ARRIVAL_DELAY*. ¿Cuáles tienen una correlación negativa?

In [ ]:
corr = vuelos[['VELOCIDAD_PROMEDIO','DISTANCE','AIR_TIME','TAXI_OUT','DEPARTURE_DELAY','ARRIVAL_DELAY']].corr()
corr.head()

5. Construya un gráfico de barras de la suma de la distancia total que recorrió cada aerolínea.

6. Los retrasos son comunmente clasificados en: _ligero retraso_ (hasta 15 min), _retraso medio_ (de 15 a 45 min) y _retraso severo_ (mayor a 45 min). Utilizando estas categorías construya un gráfico de barras del número de estos tipos de retraso de llegada para los 5 aeropuertos de destino con mayor número de vuelos.

In [ ]:
aeropuertos_principales = vuelos['DESTINATION_AIRPORT'].value_counts().reset_index().iloc[:5].DESTINATION_AIRPORT.values
aeropuertos_principales

7. ¿Qué ciudad tiene más vuelos cancelados debido al clima como una proporción de los vuelos totales por ciudad?

8. Elabore un gráfico de línea de las 5 aerolíneas con más retrasos para ver su total de retrasos por mes a lo largo del año.

In [ ]:
aerolineas_mas_retrasadas = vuelos[~(vuelos['TIPO_RETRASO'].isna())]\
    .groupby('AIRLINE').count()['DAY'].sort_values(ascending=False).iloc[:5].index